Deep artificial neural network for expressive timing and dynamics predictions in musical pieces
---------------

This notebook loads a sequential dataset with score and performance information and uses it to train and test a deep artificial neural network for generating onset timing deviation and peak loudness level of notes from musical pieces.


In [1]:
### Start Here!

import os
import numpy as np
import pandas as pd
import pickle

### Parameters to set:

runLocal = True  # set to False for using Google Colab

if runLocal:
    pathRoot = 'data/'
else:
    pathRoot = '/content/drive/My Drive/colab_data/'

#### Preparing to use TPUs and pytorch-lightning (run if using Google Colab):

In [ ]:
# Crash on purpose to get more ram :
import torch
torch.tensor([10.]*10000000000)

In [ ]:
RSION

In [ ]:
%%capture
! pip install pytorch_lightning --upgrade

#### Define the neural network

In [3]:
!wget "https://raw.githubusercontent.com/fabiozeh/deep-expression/master/dataloader.py"

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl

import dataloader as dl

np.random.seed(1728)
torch.manual_seed(1728)

class Encoder(nn.Module):
    def __init__(self, n_x, vocab_col, vocab_size, embed_size, dropout):
        super(Encoder, self).__init__()
        
        self.vocab_col = vocab_col
        self.pitchEmbedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_size, padding_idx=0)
        self.harmonyRhythmProjector = nn.Linear(in_features=n_x - 1, out_features=embed_size)
        self.drop1 = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(2*embed_size)
        self.rnn = nn.GRU(2 * embed_size, 2 * embed_size, num_layers=1, bidirectional=True)
        self.drop2 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(4*embed_size)
    
    def forward(self, x, lengths):
        pitch = torch.LongTensor(x[:, :, self.vocab_col])
        harmRhythm = torch.cat([torch.FloatTensor(x[:,:,:self.vocab_col]), torch.FloatTensor(x[:,:,self.vocab_col+1:])], dim=2)
        lengths = torch.LongTensor(lengths).view(-1)
        
        pitch = self.pitchEmbedding(pitch)
        harmRhythm = self.harmonyRhythmProjector(harmRhythm)
        src_vec = torch.cat([pitch, harmRhythm], dim=2)
        src_vec = self.norm1(self.drop1(src_vec))
        sequence = nn.utils.rnn.pack_padded_sequence(src_vec, lengths, enforce_sorted=False)
        output, _ = self.rnn(sequence)
        output, _ = nn.utils.rnn.pad_packed_sequence(output)
        return self.norm2(self.drop2(output))
    

class Decoder(nn.Module):
    def __init__(self, n_y, hidden_size, enc_hidden_size, dropout=0.1):
        super(Decoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.y_proj = nn.Linear(n_y, hidden_size)
        self.drop1 = nn.Dropout(dropout)
        self.attention = nn.MultiheadAttention(hidden_size, 1, kdim=enc_hidden_size, vdim=enc_hidden_size)
        self.rnn = nn.GRU(2*hidden_size, hidden_size, num_layers=1)
        self.drop2 = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(hidden_size)
        self.ff1 = nn.Linear(3*hidden_size, 2*hidden_size)
        self.drop3 = nn.Dropout(dropout)
        self.ff2 = nn.Linear(2*hidden_size, n_y, bias=False)
        
    def forward(self, y_prev, encoder_out, dec_hidden):
        """
        Generates outputs for a single step in the sequence (one note)
        """
        y_projected = self.drop1(self.y_proj(y_prev))  # (1, batch_size, hidden_size)
        
        # shapes: (1, b, h) <- ( (1, b, h), (len_seq, b, enc_hidden), (len_seq, b, enc_hidden) )
        context, _ = self.attention(dec_hidden, encoder_out, encoder_out)
        rnn_out, new_dec_hidden = self.rnn(torch.cat([y_projected, context], dim=2), dec_hidden)
        
        rnn_out = self.norm(self.drop2(rnn_out))
        out = self.ff2(self.drop3(F.relu(self.ff1(torch.cat([rnn_out, y_projected, context], dim=2)))))
        return out, new_dec_hidden

class Net(pl.LightningModule):

    def __init__(self, seq_length, n_x, n_y, vocab_col, vocab_size, batch_size, 
                 hidden_size, dropout_rate, output_cols, lr):
        super(Net, self).__init__()
        
        assert hidden_size % 2 == 0, "hidden_size must be multiple of 2"
        
        self.n_y = n_y
        self.seq_length = seq_length
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.output_cols = output_cols
        self.lr = lr
        self.rng = np.random.default_rng()
        
        self.encoder = Encoder(n_x, vocab_col, vocab_size, int(hidden_size/2), dropout_rate)
        self.decoder = Decoder(n_y, hidden_size, 2*hidden_size, dropout_rate)
        
        
    def forward(self, x, lengths):
        """
        Generate the entire sequence 
        """
        encoded_score = self.encoder(x, lengths)
        hidden = torch.zeros((1, x.shape[1], self.hidden_size))
        y = torch.zeros((x.shape[0], x.shape[1], self.n_y))
        prev_y = torch.zeros((1, x.shape[1], self.n_y))
        for i in range(x.shape[0]):
            prev_y, hidden = self.decoder(prev_y, encoded_score, hidden)
            y[i,:,:] = prev_y
        return y
        
    def training_step(self, batch, batch_idx):
        """
        This method doesn't use self.forward directly so we can apply teacher forcing
        on a fraction of the steps.
        """
        x, y, lengths = batch
        # encode x (score)
        encoded_score = self.encoder(x, lengths)
        
        # iterate generating y
        teacher_forcing_ratio = 0.5
        
        y = torch.FloatTensor(y)
        hidden = torch.zeros((1, x.shape[1], self.hidden_size))
        y_hat = torch.zeros((x.shape[0], x.shape[1], self.n_y))
        prev_y = torch.zeros((1, x.shape[1], self.n_y))
        for i in range(x.shape[0]):
            prev_y, hidden = self.decoder(prev_y, encoded_score, hidden)
            y_hat[i,:,:] = prev_y
            if self.rng.random() > teacher_forcing_ratio:
                prev_y = y[i,:,:].view(1, -1, self.n_y)
        loss =  F.mse_loss(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        x, y, lengths = batch
        
        y_hat = self.forward(x, lengths)
        return {'val_loss': F.mse_loss(y_hat, torch.FloatTensor(y))}

    def prepare_data(self):
        with open(os.path.join(pathRoot, 'LvB_val_sequences.data'), 'rb') as seq_path:
            self.train = pickle.load(seq_path)
        with open(os.path.join(pathRoot, 'LvB_test_sequences.data'), 'rb') as seq_path:
            self.val = pickle.load(seq_path)
        with open(pathRoot + 'LvB_pitch_dict.data', 'rb') as filehandle:
            self.lex_to_ix = pickle.load(filehandle)
            self.ix_to_lex = {v: k for k, v in self.lex_to_ix.items()}
        with open(pathRoot + 'LvB_normalizer.data', 'rb') as filehandle:
            self.moments, self.cols = pickle.load(filehandle)
            self.moments = dict(zip(self.cols, list(self.moments)))
    
    def train_dataloader(self):
        return dl.DataGenerator(self.train, self.seq_length, batch_size=self.batch_size, output_cols=self.output_cols)

    def val_dataloader(self):
        return dl.DataGenerator(self.val, self.seq_length, batch_size=self.batch_size, output_cols=self.output_cols)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.25)
        return [optimizer], [scheduler]


In [4]:
output_cols = ['ioiRatio', 'peakLevel']

model = Net(50, train[0][0][0].shape[1], len(output_cols),
            vocab_col=train[0][0][0].columns.get_loc("pitch"),
            vocab_size=len(ix_to_lex) + 3, 
            batch_size=32, 
            hidden_size=32,
            dropout_rate=0.1,
            output_cols=output_cols,
            lr=3e-3)

#### Train the model

In [5]:
trainer = pl.Trainer(num_tpu_cores=8, progress_bar_refresh_rate=20, max_epochs=1)
trainer.fit(model)

#  Save model
torch.save(model.state_dict(), pathRoot + '2021-01-29-test0.pth')

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 53 K  
1 | decoder | Decoder | 87 K  


/usr/local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


In [ ]:
# Start tensorboard.
%reload_ext tensorboard
%tensorboard --logdir lightning_logs/

#### Results

In [ ]:
# Load model
# model.load_state_dict(torch.load(pathRoot + '2021-01-11-test0.pth'))

model.eval()

#  Compute note-level error

# use validation data
# test = val

# use test data
with open(os.path.join(pathRoot, 'LvB_test_sequences.data'), 'rb') as seq_path:
    test = pickle.load(seq_path)

### CORRECT BELOW
    
def evaluation(sequences, sequence_length, model, pad_value=0.):
    Yhat = []
    for S in sequences:
        X = S[0][0]
        tx = X.shape[0]
        n_x = int(tx / sequence_length)
        n_x += 0 if tx % sequence_length == 0 else 1
        x = np.full((sequence_length, n_x, X.shape[1]), pad_value)
        for i in range(n_x - 1):            
            x[:,i,:] = X.iloc[(i * sequence_length):(i + 1) * sequence_length,:].to_numpy()
        x[:tx - (n_x - 1) * sequence_length,n_x - 1,:] = X.iloc[(n_x - 1) * sequence_length:,:].to_numpy()
        y = model(x, n_x*[sequence_length])
        print(y.shape)
        Yhat.append(y.detach().numpy().reshape((-1,y.shape[2])))
    return Yhat

def sliding_evaluation(sequences, sequence_length, model, pad_value=0., pad_start=True):
    Yhat = []
    for S in sequences:
        X = S[0]
        tx = X.shape[0]
        n_x = tx if pad_start else tx - sequence_length + 1
        x = np.full((n_x, sequence_length, X.shape[1]), pad_value)
        idx = 0
        if pad_start:
            for i in range(0, sequence_length):
                x[i,sequence_length-i-1:,:] = X.iloc[0:i+1,:].to_numpy()
            idx = sequence_length
        else:
            x[0,:,:] = X.iloc[0:sequence_length,:].to_numpy()
            idx = 1
        for i in range(1, tx - sequence_length):
            x[idx,:,:] = X.iloc[i:i+sequence_length,:].to_numpy()
            idx += 1
        y = model(x)
        if pad_start:
            Yhat.append(y[:,-1,:])
        else:
            Yhat.append(np.concatenate((y[0,:,:], y[1:, -1, :])))
    return Yhat

Yhat = evaluation(test, 200, model)
mse = np.zeros((len(test), Yhat[0].shape[1]))
ms = np.zeros((len(test), Yhat[0].shape[1]))
for i, S in enumerate(test):
    Y = S[0][1]
    Y = Y.loc[:,output_cols]
    mse[i,:] = np.mean((Yhat[i][:Y.shape[0],:] - Y) ** 2)
    ms[i,:] = np.mean(Y ** 2)
    
print('Validation set MSE for y_0: ' + str(np.mean(mse[:,0])) + '     mean square val: ' + str(np.mean(ms[:,0])))
print('Minimum y_0 MSE among pieces: ' + str(mse[:,0].min()))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(mse[:,0])
plt.plot(ms[:,0])

In [ ]:
import matplotlib.pyplot as plt

piece = 0
attr = ['peakLevel']
plt.figure(figsize=(21, 5))
plt.plot(Yhat[piece][:,0])
plt.plot(test[piece][0][1].loc[:,attr].to_numpy())
# print(test_sequences[piece][1].columns[attr])
plt.show()

#### Listen to a synthesized predicted expression

In [ ]:
import pretty_midi
import IPython.display

test_sequences = val

# piece to synthesize:
pieceNum = 27
pieceId = test[pieceNum][2]
print(pieceId)

deviations_pred = Yhat[pieceNum][:,0] * test[pieceNum][0][2][2,1] + test[pieceNum][0][2][2,0]
deviations_perf = test[pieceNum][0][1].ioiRatio * test[pieceNum][0][2][2,1] + test[pieceNum][0][2][2,0]
tempo = test[pieceNum][0][1].localTempo.iloc[0] * test[pieceNum][0][2][0,1] + test[pieceNum][0][2][0,0]
no_dev = [test[pieceNum][0][2][2,0]] * test[pieceNum][0][1].shape[0]
dev_rand = np.random.normal(size=test[pieceNum][0][1].shape[0]) * test[pieceNum][0][2][2,1] + test[pieceNum][0][2][2,0]
pm = pretty_midi.PrettyMIDI(initial_tempo=60 * tempo)
piano = pretty_midi.Instrument(1, is_drum=False, name='piano')
violin = pretty_midi.Instrument(41, is_drum=False, name='violin')
pm.instruments.append(piano)
pm.instruments.append(violin)
start = 0.
prev_note = None
for x, y, dev in zip(test[pieceNum][0][0].itertuples(), test_sequences[pieceNum][0][1].itertuples(), deviations_perf):
    (pitch, _) = ix_to_lex[x.pitch]
    start += (x.beatDiff * model.moments['beatDiff'][1] + model.moments['beatDiff'][0]) * dev
    end = start + (x.duration * model.moments['duration'][1] + model.moments['duration'][0]) * dev
    prev_note = pretty_midi.Note(100, pitch, start, end)
    if x.instrument == 'instrument_1':
        piano.notes.append(prev_note)
    else:
        violin.notes.append(prev_note)
    
IPython.display.Audio(pm.fluidsynth(fs=44100), rate=44100)

### Building conductive input from generated performance

This step uses the predicted timing information to build a local tempo signal which can be used as input in a virtual conductor. That signal is compared to the local tempo vector obtained from the chosen reference performance from the dataset.